In [1]:
from geopy.distance import geodesic
import googlemaps
from sklearn.cluster import DBSCAN
import folium
from collections import defaultdict
import requests
from os import getenv


API_KEY=getenv("GMAPS_API_KEY")

driver_zone = 2050

def calculate_centroid(passengers):
    """Calculate the centroid of a group of passengers."""
    lat = sum(p[0] for p in passengers) / len(passengers)
    lng = sum(p[1] for p in passengers) / len(passengers)
    return (lat, lng)

def interpolate_on_route(route, point):
    """Interpolate a point on the route such that it lies between two route points."""
    min_distance = float('inf')
    best_segment = None
    for i in range(len(route) - 1):
        segment_start = route[i]
        segment_end = route[i + 1]
        segment_distance = geodesic(segment_start, segment_end).kilometers

        # Project the point onto the segment
        dx, dy = segment_end[0] - segment_start[0], segment_end[1] - segment_start[1]
        projection = ((point[0] - segment_start[0]) * dx + (point[1] - segment_start[1]) * dy) / (dx * dx + dy * dy)
        projection = max(0, min(1, projection))
        interpolated_point = (
            segment_start[0] + projection * dx,
            segment_start[1] + projection * dy
        )

        # Calculate the distance from the point to the interpolated point
        distance_to_point = geodesic(point, interpolated_point).kilometers
        if distance_to_point < min_distance:
            min_distance = distance_to_point
            best_segment = interpolated_point

    return best_segment

def snap_to_road(lat, lng, api_key=API_KEY, max_retries=3, timeout=10):
    """Snap a point to the nearest road using Google Maps Roads API with retries."""
    url = f"https://roads.googleapis.com/v1/snapToRoads?path={lat},{lng}&key={api_key}&interpolate=false"
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=timeout)
            if response.status_code == 200:
                data = response.json()
                if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
                    snapped_point = data['snappedPoints'][0]['location']
                    print(f"Snapped point: ({snapped_point['latitude']}, {snapped_point['longitude']})")
                    return snapped_point['latitude'], snapped_point['longitude']
                else:
                    print(f"No snapped points returned for ({lat}, {lng}). Using fallback mechanism.")
                    return lat, lng
            else:
                print(f"Error: {response.status_code} - {response.text}. Retrying...")
        except requests.exceptions.RequestException as e:
            print(f"Request failed (attempt {attempt + 1}): {e}")
    print(f"Failed to snap to road after {max_retries} attempts. Returning original coordinates.")
    return lat, lng

def validate_snapped_point(point, route):
    """Validate and adjust snapped points to ensure they are on the route."""
    lat, lng = point
    closest_point = interpolate_on_route(route, (lat, lng))
    print(f"Validated snapped point adjusted to: {closest_point}")
    return closest_point

def adjust_pickup_point(pickup_point, passenger, distance_meters=2000):
    """Move the pickup point closer to the passenger by the specified distance."""
    total_distance = geodesic(pickup_point, passenger).meters
    if total_distance <= distance_meters:
        return pickup_point  # No adjustment needed

    ratio = distance_meters / total_distance
    adjusted_point = (
        pickup_point[0] + ratio * (passenger[0] - pickup_point[0]),
        pickup_point[1] + ratio * (passenger[1] - pickup_point[1])
    )
    return adjusted_point

def assign_clustered_pickups(route, passengers, clustering_eps=0.5):
    """
    Cluster passengers and assign a unique pickup point for each cluster such that
    the pickup point minimizes the maximum distance for all passengers in the cluster.
    Ensure pickup points are valid and on roads.
    """
    excluded_points = {route[0], route[len(route)//2], route[-1]}  # Exclude start, midpoint, and end

    # Cluster passengers based on proximity
    coords = [[p[0], p[1]] for p in passengers]
    clustering = DBSCAN(eps=clustering_eps / 111, min_samples=1, metric='euclidean').fit(coords)
    labels = clustering.labels_

    # Group passengers by cluster
    clusters = defaultdict(list)
    for passenger, label in zip(passengers, labels):
        clusters[label].append(passenger)

    assignments = {}
    pickup_points = []

    for cluster_label, cluster_passengers in clusters.items():
        # Calculate the centroid of the cluster
        centroid = calculate_centroid(cluster_passengers)

        # Interpolate the centroid onto the route
        best_pickup_point = interpolate_on_route(route, centroid)

        # Snap the pickup point to the nearest road
        snapped_point = snap_to_road(best_pickup_point[0], best_pickup_point[1])

        # Validate the snapped point
        validated_point = validate_snapped_point(snapped_point, route)

        # Assign all passengers in the cluster to the optimized pickup point
        for passenger in cluster_passengers:
            distance_to_pickup = geodesic(passenger, validated_point).meters
            if distance_to_pickup > driver_zone:
                # Move the pickup point closer to the passenger
                adjusted_pickup_point = adjust_pickup_point(validated_point, passenger, distance_meters=driver_zone)
                assignments[passenger] = adjusted_pickup_point

                # Add the adjusted pickup point to the list (ensuring uniqueness)
                if adjusted_pickup_point not in pickup_points:
                    pickup_points.append(adjusted_pickup_point)
            else:
                print(f"Passenger {passenger} is within 200m of a pickup point and will not be assigned.")

    # Debugging: Show assignment details
    for cluster_label, cluster_passengers in clusters.items():
        print(f"Cluster {cluster_label} passengers: {cluster_passengers}")
    print(f"Pickup points: {pickup_points}")

    return assignments, pickup_points

def visualize_route_and_pickups(route, passengers, assignments):
    """
    Visualize the route, passengers, and their assigned pickup points on a map.
    """
    # Create a base map centered around the first point in the route
    map_center = route[0]
    map_obj = folium.Map(location=map_center, zoom_start=13)

    # Add the route as a polyline
    folium.PolyLine(route, color="blue", weight=5, opacity=0.7, tooltip="Driver's Route").add_to(map_obj)

    # Add passenger locations
    for idx, passenger in enumerate(passengers):
        folium.Marker(
            location=passenger,
            popup=f"Passenger {idx + 1}",
            icon=folium.Icon(color="green", icon="user"),
        ).add_to(map_obj)

    # Add pickup points
    unique_pickup_points = set(assignments.values())
    for idx, pickup_point in enumerate(unique_pickup_points):
        folium.Marker(
            location=pickup_point,
            popup=f"Pickup Point {idx + 1}",
            icon=folium.Icon(color="red", icon="flag"),
        ).add_to(map_obj)

    # Draw lines between passengers and their pickup points
    for passenger, pickup_point in assignments.items():
        folium.PolyLine(
            [passenger, pickup_point],
            color="purple",
            weight=2,
            opacity=0.6,
            tooltip=f"Assigned to Pickup Point"
        ).add_to(map_obj)

    # Return the map object for display
    return map_obj

# Example Usage
route = [
    
]

passengers = [
]

# Assign clustered pickup points
assignments, pickup_points = assign_clustered_pickups(route, passengers, clustering_eps=0.5)


Optimized Route for Driver John Doe:
  Stop 1: (50.8467, 4.3524)
  Stop 2: (50.8804, 4.4724)
  Stop 3: (51.2194, 4.4025)

Optimized Route for Driver Liam Gray:
  Stop 1: (51.215, 4.419)
  Stop 2: (51.2194, 4.4025)
  Stop 3: (51.2248, 4.4353)

Optimized Route for Driver Oliver Brown:
  Stop 1: (51.2196, 4.4377)
  Stop 2: (51.2194, 4.4025)

Optimized Route for Driver Lucas Van Dam:
  Stop 1: (51.0543, 3.7174)
  Stop 2: (51.0602, 3.7308)
  Stop 3: (51.2194, 4.4025)

Optimized Route for Driver Marie Lemmens:
  Stop 1: (51.1316, 4.5728)
  Stop 2: (51.1253, 4.5722)
  Stop 3: (51.2194, 4.4025)


Assigned Rides:
Passenger: Alice assigned to Driver: John
Passenger: Sophia assigned to Driver: Liam
Passenger: Sophie assigned to Driver: Lucas
Passenger: Eva assigned to Driver: Marie
